## Инструменты для анализа контекстно-свободных грамматик

#### Клышинский Э.С.
#### 21.05.2022

Общая идея компиляторов компиляторов заключется в следующем. Обычный компилятор берет текст программы и переводит его, например, в машинный код. Компилатор компиляторов берет грамматику языка и перевод его в исходный код компилятора или интерпретатора данного языка, или в программу, которая просто проверяет правильность выражения.

## LARK

Для начала познакомимся с простой библиотекой Lark, имеющей ограниченный функционал. Класс из этой библиотеки принимает на вход грамматику в некотором формате. Далее она получает на вход строку и проверяет ее на соответствие этой грамматике.

Хороший обзор разных библиотек находится [здесь](https://tomassetti.me/parsing-in-python/).

К плюсам Lark относится [наличие](https://www.lark-parser.org/ide/) отладчика.

In [1]:
# Установим библиотеку.
!pip3 install lark

Defaulting to user installation because normal site-packages is not writeable


Пример, на котором мы будем разбирать работу библиотеки взят [отсюда](http://blog.erezsh.com/how-to-write-a-dsl-in-python-with-lark/) (но он также есть и в [документации](https://lark-parser.readthedocs.io/en/latest/examples/turtle_dsl.html)). Для него нам потребуется библиотека рисования `turtle`, которая требует установленного `tkInter`.

In [2]:
!sudo apt-get install python3-tk

[sudo] пароль для edward: 


Существует два принципиально разных подхода к написанию компиляторов компиляторов. Подход постарше заключается в написании грамматики, подход помладше состоит в написании кода на нужном вам языке программирования, который будет использоваться библиотекой для разбора. Мы с вами пройдем по классике.

Так исторически сложилось, что грамматики для разных инструментов пишутся в примерно одинаковой нотации, различающейся особенностями работы. Выделяют описания лексем (единиц языка) и описания правил (в терминах БНФ). Для обоих видов сперва идет название, потом двоеточие, и после него - описание. Традиционно, названия лексем пишутся большими буквами, названия правил начинаются с маленькой буквы. Например,  
`COLOR: ("a".."z")+`  
описывает лексему `COLOR`, описывающуюся при помощи регулярного выражения `("a".."z")+`. Правило  
`code_block: "{" instruction+ "}"`  
описывает правило `code_block`, состоящее из положительной итерации строк, описываемых правилом `instruction`, обрамленных фигурными скобками.
Обратите внимание, что регулярные выражения библиотеки Lark имеют собственную нотацию.

Положим грамматику в переменную, чтобы потом передать библиотеке.

In [1]:
turtle_grammar = '''start: instruction+
 
instruction: ("f"|"b"|"l"|"r") NUMBER
           | "c" COLOR [COLOR]
           | "fill" code_block
           | "repeat" NUMBER code_block
 
code_block: "{" instruction+ "}"
 
COLOR: ("a".."z")+
NUMBER: ("0".."9")+
WHITESPACE: (" " | "\\n")+
%ignore WHITESPACE
'''

Напишем программу, которую мы собираемся выполнять при помощи библиотеки Lark.

In [2]:
text = """
c red yellow
fill { repeat 36 {
    f200 l170
}}
"""

Импортируем саму библиотеку.

In [3]:
from lark import Lark
import lark

Создадим объект для разбора программ на нашем языке и передадим в него грамматику языка.

In [4]:
parser = Lark(turtle_grammar)  # Scannerless Earley is the default

Посмотрим что возвращает функция разбора.

In [5]:
t = parser.parse(text)
print(t)

Tree(Token('RULE', 'start'), [Tree(Token('RULE', 'instruction'), [Token('COLOR', 'red'), Token('COLOR', 'yellow')]), Tree(Token('RULE', 'instruction'), [Tree(Token('RULE', 'code_block'), [Tree(Token('RULE', 'instruction'), [Token('NUMBER', '36'), Tree(Token('RULE', 'code_block'), [Tree(Token('RULE', 'instruction'), [Token('NUMBER', '200')]), Tree(Token('RULE', 'instruction'), [Token('NUMBER', '170')])])])])])])


Ниже дается прямая и обратная польская запись выражений.

In [ ]:
5-3-1
-(-(5, 3), 1)
((5,3)-, 1)-
(1)

((5,3)pow, (42, 12)-)-
[95]

Ниже нарисована пара абстрактных синтаксических деревьев для вычисления арифметических выражений и для генератора.

In [ ]:
5 - 3 - 1

     -
  -     1
5  3

1 - (5 - 3)
     -
  1     -    
       5  3


[f(i) for i in range(10, 20, 2)]

      [for]
   /    |      \
  i  range     f()
    /  | \     |
    10 20 2    i

    for
  /  |  \
i   f2   f
     |   |
     +   i
    / \       
   j   3 

Судя по нотации, она возвращает дерево, и это дерево разбора программы. Это же дерево можно нарисовать покрасивее средствами самой библиотеки.

In [6]:
print(parser.parse(text).pretty())

start
  instruction
    red
    yellow
  instruction
    code_block
      instruction
        36
        code_block
          instruction	200
          instruction	170



А можно нарисовать еще красивее при помощи вот [этой](https://github.com/klyshinsky/ipytreewidget) библиотечки.

In [7]:
from ipytreewidget import TreeWidget

In [8]:
def shape_lark_tree(tree):
    '''Функция формирует дерево разбора в виде, понятном библиотеке отрисовки дерева.'''
    if type(tree) is lark.lexer.Token:
        dct = {"text": tree.type + ': ' + tree.title(), "text-color":"#A64500"}
    elif type(tree) is lark.tree.Tree:
        dct = {"text": str(tree.data), "text-color":"#188A69"}
        childs = []
        for child in tree.children:
            childs.append(shape_lark_tree(child))
        if childs != []:
            dct["childs"] = childs
    return dct

tree = parser.parse(text)
tr2 = TreeWidget()
tr2.show(shape_lark_tree(tree), "<b>Parse Tree</b>", footer="End of Tree")

HTML(value='<div style="border: 1px solid #DDDDDD"><div style="width:100%;background-color:#DDDDDD;padding-lef…

Усложним грамматику, и вместо того, чтобы самим определять базовые лексемы, импортируем их из библиотеки.  
Помимо этого, дадим уникальное имя для каждой продукции, чтбы различать какое из нескольких действий было совершено. Это сделает дерево разбора более читаемым: по нему будет понятно, что именно надо сделать.

In [9]:
turtle_grammar = '''
start: instruction+
 
instruction: MOVEMENT NUMBER            -> movement
           | "c" COLOR [COLOR]          -> change_color
           | "fill" code_block          -> fill
           | "repeat" NUMBER code_block -> repeat
 
code_block: "{" instruction+ "}"
 
MOVEMENT: "f"|"b"|"l"|"r"
COLOR: LETTER+
 
%import common.LETTER
%import common.INT -> NUMBER
%import common.WS
%ignore WS
'''

In [10]:
parser = Lark(turtle_grammar)
tree = parser.parse(text)
tr2 = TreeWidget()
tr2.show(shape_lark_tree(tree), "<b>Parse Tree</b>", footer="End of Tree")

HTML(value='<div style="border: 1px solid #DDDDDD"><div style="width:100%;background-color:#DDDDDD;padding-lef…

Импортируем библиотеку `turtle` и выполним получившееся дерево.

Вместо того, чтобы писать собственную функцию можно использовать [шаблон "Слушатель"](https://lark-parser.readthedocs.io/en/latest/visitors.html).

In [11]:
import turtle

In [12]:
def run_instruction(t):
    if t.data == 'change_color':
        turtle.color(*t.children)   # We just pass the color names as-is
 
    elif t.data == 'movement':
        name, number = t.children
        {
            'f': turtle.fd,
            'b': turtle.bk,
            'l': turtle.lt,
            'r': turtle.rt,
        }[name](int(number))
 
    elif t.data == 'repeat':
        count, block = t.children
        for i in range(int(count)):
            run_instruction(block)
 
    elif t.data == 'fill':
        turtle.begin_fill()
        run_instruction(t.children[0])
        turtle.end_fill()
 
    elif t.data == 'code_block':
        for cmd in t.children:
            run_instruction(cmd)
 
    else:
        raise SyntaxError('Unknown instruction: %s' % t.data)

In [13]:
for inst in tree.children:
    run_instruction(inst)

Terminator: 

Вот [здесь](https://github.com/lark-parser/lark/tree/master/examples) есть еще немного примеров.

-----

## ANTLR

Теперь посмотрим на другую библиотеку, устроенную несколько иначе - ANTLR.  
Для этой библиотеки проще отдельно оформить файл с грамматикой. Сама грамматику устроена более сложно и в нее можно добавлять фрагменты собственного кода, которые будут вызываться по ходу разбора. Используя утилиту из командной строки, мы генерируем исходный код программы (в нашем случае на Питоне). Далее этот исходный код можно импортировать в свою собственную программу и инициировать разбор нужной нам строки.

Инсталлируем всё необходимое программное обеспечение [отсюда](https://github.com/antlr/antlr4/blob/master/doc/getting-started.md) (или [отсюда](https://faun.pub/introduction-to-antlr-python-af8a3c603d23), если у вас Линукс хотя разница в инструкциях на одну Java) (или просто скачиваем [отсюда](https://www.antlr.org/download). Эти ссылки можно использовать и как введение самого начального уровня. Полная документация находится [здесь](https://github.com/antlr/antlr4/blob/4.6/doc/index.md). Хорошее и полное введение [здесь](https://tomassetti.me/antlr-mega-tutorial/).


    cd /usr/local/lib
    curl -O https://www.antlr.org/download/antlr-4.13.1-complete.jar
    export CLASSPATH=”.:/usr/local/lib/antlr-4.5-complete.jar:$CLASSPATH” 
    # (Exports class path. Include in .bashrc file)
    alias antlr4=’java -Xmx500M -cp “/usr/local/lib/antlr-4.5-complete.jar:$CLASSPATH” org.antlr.v4.Tool’ 
    # (Creates alias for ANTLR tool. Include in .bashrc file)
    alias grun=’java org.antlr.v4.gui.TestRig’ 
    # (Creates alias for TestRig. Include in .bashrc file)
    # Restart the machine

In [2]:
!pip install antlr4-python3-runtime

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 KB 1.2 MB/s eta 0:00:00m eta 0:00:010:00:01


In [14]:
# Импортируем необходимые библиотеки.
import sys
from antlr4 import *

In [32]:
# Переходим в каталог с грамматиками и запускаем утилиту, написанную на Java, для генерации исходного кода.
!cd antlr_grammars && \
java -Xmx500M -cp "/usr/local/lib/antlr-4.13.1-complete.jar" org.antlr.v4.Tool -Dlanguage=Python3 -o outputs T.g4

Посмотрим на параметры запуска утилиты.  
`java -Xmx500M -cp "/usr/local/lib/antlr-4.10.1-complete.jar" org.antlr.v4.Tool`  
Это мы запускаем программу на Java.  
`-Dlanguage=Python3` - это мы просим ее компилировать выход для языка Питон.  
`-o outputs` - это мы просим ее складывать все сгенерированные файлы в каталог `outputs`.  <br>
`T.g4` - это мы просим ее сгенерировать код для грамматики из файла `T.g4`.

Теперь посмотрим что нам сгенерировала утилита в каталоге `outputs`. Среди прочих файлов, в которых содержатся символы грамматики, обратим внимание на следующие.
`TLexer.py` - файл с исходными кодами для лексического анализатора.  
`TParser.py` - файл с исходными кодами для синтаксического анализатора.  
`TListener.py` - файл с базовым классом, который обеспечивает обход дерева разбора.

Постфикс в имени файлов стандартный, префикс совпадает с именем файла, содержащего грамматику.  

Посмотрим на простую грамматику, разбирающую арифметические выражения в return. **Название грамматики в первой строке обязано совпадать с именем файла!**

In [33]:
!more antlr_grammars/T.g4

grammar T;

stat: 'return ' e ';' # Return
 	| 'break' ';' # Break
 	;
e   : e '*' e # Mult
    | e '+' e # Add
    | INT # Int
    | ID # Id
    ;

WS : [ \r\t\n]+ ;
INT: [0-9]+;
ID: [a-zA-Z][a-zA-Z0-9]*;



Лексемы описываются при помощи обычных регулярных выражений. После решетки пишется название для конкретной продукции. ANTLR использует ее для того чтобы генерировать названия функций в классе Listener. На этом уровне синтаксис правил практически не отличается от того, что мы видеи у Lark.

Файлы созданы, можно импортировать из них необходимые классы. Но так как Питон считает, что импортировать достаточно только один раз, будем сперва выгружать старые версии библиотек, а потом загружать их новую версию.

In [34]:
if 'antlr_grammars.outputs.TLexer' in sys.modules:  
    del sys.modules['antlr_grammars.outputs.TLexer']
    del sys.modules['antlr_grammars.outputs.TParser']
    del sys.modules['antlr_grammars.outputs.TListener']
from antlr_grammars.outputs.TLexer import TLexer
from antlr_grammars.outputs.TParser import TParser
from antlr_grammars.outputs.TListener import TListener

Создаем входной поток в зависимости от того, откуда мы хотим получить входную последовательность (`FileStream` для файла или `InputStream` для строки). Создаем объект для лексического анализа, отдаем его потоку для токенов. Создаем класс для синтаксического анализа `TParser`. Для него вызываем функцию с тем же именем, что и имя начального символа нашей грамматики.

In [35]:
# infile = FileStream('antlr_grammars/T_in.txt')
infile = InputStream(' return 1*2+asdf;')
infile.consume()
lexer = TLexer(infile)
stream = CommonTokenStream(lexer)
parser = TParser(stream)
tree = parser.stat()

Теперь переопределим некоторые функции у класса TListener с тем, чтобы они делали нужные нам действия: вывод абстрактного синтаксического дерева для заданного выражения.

Сравнение "Слушателя" с "Посетителем" [здесь](https://habr.com/ru/post/259691/).

In [ ]:
-
  -
    5
    3
  1

In [36]:
class MyListener(TListener):

    def __init__(self):
        self.shift = ''
        self.shiftSize = 2

    # Enter a parse tree produced by TParser#Add.
    def enterAdd(self, ctx:TParser.AddContext):
        print(self.shift + '+')
        self.shift += ' ' * self.shiftSize

    # Exit a parse tree produced by TParser#Add.
    def exitAdd(self, ctx:TParser.AddContext):
        self.shift = self.shift[:-self.shiftSize]


    # Enter a parse tree produced by TParser#Mult.
    def enterMult(self, ctx:TParser.MultContext):
        print(self.shift + '*')
        self.shift += ' ' * self.shiftSize

    # Exit a parse tree produced by TParser#Mult.
    def exitMult(self, ctx:TParser.MultContext):
        self.shift = self.shift[:-self.shiftSize]

    # Enter a parse tree produced by TParser#Id.
    def enterId(self, ctx:TParser.IdContext):
        print(self.shift + ctx.getText())

    # Enter a parse tree produced by TParser#Int.
    def enterInt(self, ctx:TParser.IntContext):
        print(self.shift+ctx.getText())



Класс `ParseTreeWalker` позволяет "гулять" по дереву разбора и выполнять нужные действия.

In [37]:
printer = MyListener()
walker = ParseTreeWalker()
walker.walk(printer, tree)

+
  *
    1
    2
  asdf


Изменим нашу грамматику, добавив в нее массу новых возможностей.

Ключ `@header` позволяет добавить код, который будет помещен в начало файла с исходным кодом класса синтаксического анализатора.

Ключ `@members` добавит члены в класс синтаксического анализатора. Теперь мы можем пользоваться нужными нам библиотеками и членами класса анализатора в ходе разбора.

В фигурных скобках можно добавить исходный код, который будет выполняться сразу как только анализатор разберет правило до него. В ключе `@after` можно добавить код, который выполнится в случае успешного разбора цепочки при помощи правила. В добавляемом исходном коде можно исползовать свойства текущего правила или токена, добавив к нему в начале символ доллара. 

Секция `locals` после имени файла позволяет завести локальные переменные, к которым можно обращаться в коде, размещенном в вызываемых правилах. При повторном (например, рекурсивном) посещении этого правила переменные будут создаваться повторно.


In [38]:
!cat antlr_grammars/T2.g4

grammar T2;

@header {
import math
}
 
@members {
res = ""
}

// Rule stat.
stat locals [stack=[]]    :
          'return ' e ';' 
          {print("calculated:", $stack)} # Return
 	| 'break' ';' # Break
 	;

// Rule e.
e   
@after{print("processed: ", $stat::stack);}
    : e '*' e 
      {$stat::stack[-2]*=$stat::stack[-1]; $stat::stack.pop(); print("* ", end=''); 
      } # Mult
    | e '+' e 
      {$stat::stack[-2]+=$stat::stack[-1]; $stat::stack.pop(); print("+ ", end=''); 
      } # Add
    | 'pow(' INT {a=int($INT.text)} ',' INT {b=int($INT.text)} ')'
      {
$stat::stack.append(math.pow(a,b)); print("pow ", end=''); 
      } # Pow
    | INT 
      {
self.res+=$text; 
$stat::stack.append(int($text)); 
print("->", $stat::stack)
      } # Int
    | ID # Id
    ;

// Lexer rules after.
INT: [0-9]+;
ID: [a-zA-Z][a-zA-Z0-9]*;
WS : [ \r\t\n]+ -> skip;



In [42]:
# !cd antlr_grammars && java -Xmx500M -cp "/usr/local/lib/antlr-4.10.1-complete.jar:$CLASSPATH" org.antlr.v4.Tool -Dlanguage=Python3 -o outputs T2.g4
!cd antlr_grammars && \
java -Xmx500M -cp "/usr/local/lib/antlr-4.13.1-complete.jar" org.antlr.v4.Tool -Dlanguage=Python3 -o outputs T2.g4

In [43]:
if 'antlr_grammars.outputs.T2Lexer' in sys.modules:  
    del sys.modules['antlr_grammars.outputs.T2Lexer']
if 'antlr_grammars.outputs.T2Parser' in sys.modules:  
    del sys.modules['antlr_grammars.outputs.T2Parser']
from antlr_grammars.outputs.T2Lexer import T2Lexer
from antlr_grammars.outputs.T2Parser import T2Parser

In [44]:
infile = InputStream(' return 10 * 2 + pow(3, 4) * 2;')
infile.consume()
lexer2 = T2Lexer(infile)
stream = CommonTokenStream(lexer2)
parser2 = T2Parser(stream)
tree = parser2.stat()

-> [10]
-> [10, 2]
processed:  [10, 2]
* pow -> [20, 81.0, 2]
processed:  [20, 81.0, 2]
* processed:  [20, 162.0]
+ processed:  [182.0]
calculated: [182.0]


In [45]:
parser2.res

'1022'

Важен порядок следования операций в правиле!

In [46]:
!cat antlr_grammars/T3_incorrect.g4

grammar T3_incorrect;

@header {
import math
}
 
@members {
res = ""
}

// Rule stat.
stat locals [stack=[]]    :
          'return ' e ';' 
          {print("calculated:", $stack)} # Return
 	| 'break' ';' # Break
 	;

// Rule e.
e   
@after{print("processed", $stat::stack);}
    : e '*' e 
      {$stat::stack[-2]*=$stat::stack[-1]; $stat::stack.pop(); print("*"); 
      } # Mult
    | e '/' e 
      {$stat::stack[-2]/=$stat::stack[-1]; $stat::stack.pop(); print("/"); 
      } # Div
    | e '+' e 
      {$stat::stack[-2]+=$stat::stack[-1]; $stat::stack.pop(); print("+"); 
      } # Add
    | e '-' e 
      {$stat::stack[-2]-=$stat::stack[-1]; $stat::stack.pop(); print("-|"); 
      } # Sub
    | 'pow(' INT {a=int($INT.text)} ',' INT {b=int($INT.text)} ')'
      {
$stat::stack.append(math.pow(a,b)); print("pow"); 
      } # Pow
    | INT 
      {
self.res+=$text; 
$stat::stack.append(int($text)); 
print("->", $stat::stack)
      } # Int
    | ID # Id
    | '(' e ')' # Braced
    ;

// 

In [47]:
# !cd antlr_grammars && java -Xmx500M -cp "/usr/local/lib/antlr-4.10.1-complete.jar:$CLASSPATH" org.antlr.v4.Tool -Dlanguage=Python3 -o outputs T2.g4
!cd antlr_grammars && \
java -Xmx500M -cp "/usr/local/lib/antlr-4.13.1-complete.jar" org.antlr.v4.Tool -Dlanguage=Python3 -o outputs T3_incorrect.g4

In [48]:
if 'antlr_grammars.outputs.T3_incorrectLexer' in sys.modules:  
    del sys.modules['antlr_grammars.outputs.T3_incorrectLexer']
if 'antlr_grammars.outputs.T3_incorrectParser' in sys.modules:  
    del sys.modules['antlr_grammars.outputs.T3_incorrectParser']
from antlr_grammars.outputs.T3_incorrectLexer import T3_incorrectLexer
from antlr_grammars.outputs.T3_incorrectParser import T3_incorrectParser

In [49]:
infile = InputStream(' return 16/2/2-(pow(3,4)-2*3)-5*6;')
infile.consume()
lexer3_i = T3_incorrectLexer(infile)
stream = CommonTokenStream(lexer3_i)
parser3_i = T3_incorrectParser(stream)
tree = parser3_i.stat()

-> [16]
-> [16, 2]
processed [16, 2]
/
-> [8.0, 2]
processed [8.0, 2]
/
pow
-> [4.0, 81.0, 2]
-> [4.0, 81.0, 2, 3]
processed [4.0, 81.0, 2, 3]
*
processed [4.0, 81.0, 6]
-|
processed [4.0, 75.0]
processed [4.0, 75.0]
-|
-> [-71.0, 5]
-> [-71.0, 5, 6]
processed [-71.0, 5, 6]
*
processed [-71.0, 30]
-|
processed [-101.0]
calculated: [-101.0]


In [50]:
class MyListener3(TListener):

    def __init__(self):
        self.shift = ''
        self.shiftSize = 3

    # Enter a parse tree produced by TParser#Add.
    def enterAdd(self, ctx:TParser.AddContext):
        print(self.shift + '+')
        self.shift += ' ' * self.shiftSize

    # Exit a parse tree produced by TParser#Add.
    def exitAdd(self, ctx:TParser.AddContext):
        self.shift = self.shift[:-self.shiftSize]

    # Enter a parse tree produced by TParser#Add.
    def enterSub(self, ctx:TParser.AddContext):
        print(self.shift + '-')
        self.shift += ' ' * self.shiftSize

    # Exit a parse tree produced by TParser#Add.
    def exitSub(self, ctx:TParser.AddContext):
        self.shift = self.shift[:-self.shiftSize]

    # Enter a parse tree produced by TParser#Mult.
    def enterMult(self, ctx:TParser.MultContext):
        print(self.shift + '*')
        self.shift += ' ' * self.shiftSize

    # Exit a parse tree produced by TParser#Mult.
    def exitMult(self, ctx:TParser.MultContext):
        self.shift = self.shift[:-self.shiftSize]

    # Enter a parse tree produced by TParser#Add.
    def enterDiv(self, ctx:TParser.AddContext):
        print(self.shift + '/')
        self.shift += ' ' * self.shiftSize

    # Exit a parse tree produced by TParser#Add.
    def exitDiv(self, ctx:TParser.AddContext):
        self.shift = self.shift[:-self.shiftSize]
        
    # Enter a parse tree produced by TParser#Id.
    def enterId(self, ctx:TParser.IdContext):
        print(self.shift + ctx.getText())

    # Enter a parse tree produced by TParser#Int.
    def enterInt(self, ctx:TParser.IntContext):
        print(self.shift+ctx.getText())

    def enterPow(self, ctx:TParser.IdContext):
        print(self.shift + ctx.getText())

        

In [51]:
printer = MyListener3()
walker = ParseTreeWalker()
walker.walk(printer, tree)
# 16/2/2-(pow(3,4)-2*3)+5*6;

-
   -
      /
         /
            16
            2
         2
      -
         pow(3,4)
         *
            2
            3
   *
      5
      6


In [52]:
!cat antlr_grammars/T3_correct.g4

grammar T3_correct;

@header {
import math

def make_op(stack, op):
    if op == '*': 
        stack[-2] *= stack[-1] 
    elif op == '/':
        stack[-2] /= stack[-1]
    elif op == '+':
        stack[-2] += stack[-1]
    elif op == '-':
        stack[-2] -= stack[-1]

    stack.pop()
    print(op)
}
 
@members {
res = ""
}

// Rule stat.
stat locals [stack=[]]    :
          'return ' e ';' 
          {print("calculated:", $stack)} # Return
 	| 'break' ';' # Break
 	;

// Rule e.
e   
@after{print("processed", $stat::stack);}
    : e OP_MUL e 
      {make_op($stat::stack, $OP_MUL.text)
      } # Mult
    | e OP_ADD e 
      {make_op($stat::stack, $OP_ADD.text)
      } # Add
    | 'pow(' INT {a=int($INT.text)} ',' INT {b=int($INT.text)} ')'
      {
$stat::stack.append(math.pow(a,b)); print("pow"); 
      } # Pow
    | INT 
      {
self.res+=$text; 
$stat::stack.append(int($text)); 
print("->", $stat::stack)
      } # Int
    | ID # Id
    | '(' e ')' # Braced
    ;

// Lexer rules a

In [53]:
# !cd antlr_grammars && java -Xmx500M -cp "/usr/local/lib/antlr-4.10.1-complete.jar:$CLASSPATH" org.antlr.v4.Tool -Dlanguage=Python3 -o outputs T2.g4
!cd antlr_grammars && \
java -Xmx500M -cp "/usr/local/lib/antlr-4.13.1-complete.jar" org.antlr.v4.Tool -Dlanguage=Python3 -o outputs T3_correct.g4

In [54]:
if 'antlr_grammars.outputs.T3_correctLexer' in sys.modules:  
    del sys.modules['antlr_grammars.outputs.T3_correctLexer']
if 'antlr_grammars.outputs.T3_correctParser' in sys.modules:  
    del sys.modules['antlr_grammars.outputs.T3_correctParser']
from antlr_grammars.outputs.T3_correctLexer import T3_correctLexer
from antlr_grammars.outputs.T3_correctParser import T3_correctParser

In [55]:
infile = InputStream(' return 16/2/2-(pow(3,4)-2*3)-5*6;')
infile.consume()
lexer3_c = T3_correctLexer(infile)
stream = CommonTokenStream(lexer3_c)
parser3_c = T3_correctParser(stream)
tree = parser3_c.stat()

-> [16]
-> [16, 2]
processed [16, 2]
/
-> [8.0, 2]
processed [8.0, 2]
/
pow
-> [4.0, 81.0, 2]
-> [4.0, 81.0, 2, 3]
processed [4.0, 81.0, 2, 3]
*
processed [4.0, 81.0, 6]
-
processed [4.0, 75.0]
processed [4.0, 75.0]
-
-> [-71.0, 5]
-> [-71.0, 5, 6]
processed [-71.0, 5, 6]
*
processed [-71.0, 30]
-
processed [-101.0]
calculated: [-101.0]


In [56]:
!cat antlr_grammars/T4.g4

grammar T4;

@header {
import math

def make_op(stack, op):
    if op == '*': 
        stack[-2] *= stack[-1] 
    elif op == '/':
        stack[-2] /= stack[-1]
    elif op == '+':
        stack[-2] += stack[-1]
    elif op == '-':
        stack[-2] -= stack[-1]

    stack.pop()
    print(op)
}
 
@members {
values = {}
}

// Rule start.
start:
      (decl | a_expr)+
;

// Rule decl.
decl:
      'int' ID {self.values[$ID.text]=0} (',' ID {self.values[$ID.text]=0} )* ';' WS*
{print('delcared', $text)
}   
;

// Rule a_expr.
a_expr locals [stack=[]]    :
      ID {if $ID.text not in self.values.keys(): print(f"wasn't declared:{$ID.text}")} '=' e ';' WS*
      {print("calculated:", $stack);self.values[$ID.text]=$stack[-1];} # Return
    | 'break' ';' # Break
;

// Rule e.
e   
@after{print("processed", $a_expr::stack);}
    : e OP_MUL e 
      {make_op($a_expr::stack, $OP_MUL.text)
      } # Mult
    | e OP_ADD e 
      {make_op($a_expr::stack, $OP_ADD.text)
      } # Add
    | 'pow(' INT

In [57]:
# !cd antlr_grammars && java -Xmx500M -cp "/usr/local/lib/antlr-4.10.1-complete.jar:$CLASSPATH" org.antlr.v4.Tool -Dlanguage=Python3 -o outputs T2.g4
!cd antlr_grammars && \
java -Xmx500M -cp "/usr/local/lib/antlr-4.13.1-complete.jar" org.antlr.v4.Tool -Dlanguage=Python3 -o outputs T4.g4

In [58]:
if 'antlr_grammars.outputs.T4Lexer' in sys.modules:  
    del sys.modules['antlr_grammars.outputs.T4Lexer']
if 'antlr_grammars.outputs.T4Parser' in sys.modules:  
    del sys.modules['antlr_grammars.outputs.T4Parser']
from antlr_grammars.outputs.T4Lexer import T4Lexer
from antlr_grammars.outputs.T4Parser import T4Parser

In [59]:
infile = InputStream(' int id1,id2;id1=16/2/2-(pow(3,4)-2*3)+5*3;id2=id1*2;')
infile.consume()
lexer4 = T4Lexer(infile)
stream = CommonTokenStream(lexer4)
parser4 = T4Parser(stream)
tree = parser4.start()

delcared int id1,id2;
-> [16]
-> [16, 2]
processed [16, 2]
/
-> [8.0, 2]
processed [8.0, 2]
/
pow
-> [4.0, 81.0, 2]
-> [4.0, 81.0, 2, 3]
processed [4.0, 81.0, 2, 3]
*
processed [4.0, 81.0, 6]
-
processed [4.0, 75.0]
processed [4.0, 75.0]
-
-> [-71.0, 5]
-> [-71.0, 5, 3]
processed [-71.0, 5, 3]
*
processed [-71.0, 15]
+
processed [-56.0]
calculated: [-56.0]
-> [-56.0, 2]
processed [-56.0, 2]
*
processed [-112.0]
calculated: [-112.0]


line 1:3 extraneous input ' ' expecting ID


In [ ]:
parser4.values

{'id1': -56.0, 'id2': -112.0}